In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px

from sklearn.datasets import make_classification
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
#Import in Absenteeism at work data
absent_df = pd.read_csv("Absenteeism_at_work.csv",";")
absent_df.head()

<ipython-input-2-500933cae626>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  absent_df = pd.read_csv('Downloads','Absenteeism_at_work_AAA', 'Absenteeism_at_work.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'Downloads'

In [ ]:
absent_df.info()

In [ ]:
 # Assign X  (data) multiple linear regression

X = absent_df[['Age','Day of the week','Reason for absence','Seasons','Service time',
              'Hit target','Education','Son','Weight','Body mass index']]
y = absent_df['Absenteeism time in hours']
print(X.shape, y.shape)


In [ ]:
 # Split the data into training and testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:
# Create a StandardScaler() model and fit it to the training data
scaler = StandardScaler().fit(X_train)

# Transform the training and testing data by using the X_scaler and y_scaler models
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Create, fit and evaluate decision tree model 
clf = DecisionTreeClassifier().fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

# Print the confusion matrix
print(classification_report(y_test, y_pred))

In [ ]:
# Print scores
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

In [ ]:
 # Create a LinearRegression model and fit it to the scaled training data
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)


In [ ]:
# Make predictions using a fitted model
predictions = model.predict(X_test)
pd.DataFrame({"Predictions" : predictions}).head()


In [ ]:
# Plot the difference between the model predicted values and actual y values, versus the model predicted values
# predicted values of X training using the model.predict() method 

plt.scatter(model.predict(X_train), model.predict(X_train) - y_train, c="blue", label="Training Data")
plt.scatter(predictions, predictions - y_test, c="red", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=predictions.min(), xmax=predictions.max())
plt.title("Residual Plot")


In [ ]:
# Created DataFrame of results
predictions_df = pd.DataFrame({"Predicted": predictions, "Actual": y_test, "Residual": predictions - y_test})
predictions_df.head(10)

In [ ]:
#Random Forests
clfs = []
scores = []

for i in range(50):
    
    # Sample the data for each new tree
    X_train_scaled_bootstrap, y_train_bootstrap = resample(X_train_scaled, y_train, random_state=i)
    
    # Create a decision tree and append it to our list of classifiers
    clf = DecisionTreeClassifier(random_state=i+200).fit(X_train_scaled_bootstrap, y_train_bootstrap)
    clfs.append(clf)
    
    # Take the median score of all the created classifiers
    y_preds = [clf.predict(X_test_scaled) for clf in clfs]
    y_pred = pd.DataFrame(y_preds).median().round()
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

In [ ]:
 # Visualize the bootstrap decision trees 
plt = px.line(scores,labels={"index":"Bootstrap Iteraton","value":"Testing Accuracy"})
plt.layout.update(showlegend=False)
plt.show()

In [ ]:
 # Fit a model and print training and testing scores
clf = RandomForestClassifier(random_state=1, n_estimators=50).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')